In [64]:
from IPython.core.pylabtools import figsize
from dateutil.relativedelta import relativedelta
from dateutil.utils import today
from eolearn.core import (
    FeatureType,
    SaveTask,
    linearly_connect_tasks,
    EOWorkflow,
    EOExecutor,
    OutputTask,
)
from eolearn.io import get_available_timestamps, SentinelHubInputTask
from sentinelhub import SHConfig, BBox, CRS, DataCollection
import matplotlib.pyplot as plt
from eolearn.core import EOPatch
import numpy as np
import geopandas as gpd
from pathlib import Path
from importlib import reload
from matplotlib import pyplot as plt

In [65]:
config = SHConfig(profile="sentinel-dl")

# Resolution (resolution), resolution (size) and resolution (bbox size).
Sounds as confusing as it is...
in this case `resolution` refers to sentinel sampling resolution in meters (usually 10m). Size refers to resolution of patch in pixels. Then bbox also contains size in meters of earth covered.

It's important to be careful when setting these up.
Side note: `SentinelHubInputTask` can't take simultaneous `size` and `resolution`, so in order to get let's say region of 2560m with resolution of 10m we need bbox with 2560m side and pass resolution=10. 
The problem is if you want to have a region of 5120m with resolution of 20m and patch of 512x512pixels, because this will default to 256x256 (as that is the size if you sample 5120m with spatial resolution of 20m).
Now one can try this: specify 5120m sided bbox and set size to 512 (but you can't also specify resolution). This then yields 512x512 res just fine, but it doesn't say which sampling resolution it uses (probably 10). 

In [66]:
def test(r, p, f, force_res=None):
    resolution = r
    patch_size = p
    collection = DataCollection.SENTINEL2_L1C

    import util.region as rt
    from util.region import prepare_slo_chunks

    gdf, bbox_list = prepare_slo_chunks(
        resolution=resolution, patch_size=patch_size, fixed_meter_patch_size=f
    )
    if force_res is None:
        input_task = SentinelHubInputTask(
            data_collection=collection,
            bands=["B01", "B02", "B03"],
            bands_feature=(FeatureType.DATA, "L1C_data"),
            additional_data=[(FeatureType.MASK, "dataMask")],  # cloud mask
            maxcc=0.8,
            size=(
                patch_size,
                patch_size,
            ),
            config=config,  # important since we are using sentinel-dl, alternatively save ID and secret to default profile
        )
    else:
        input_task = SentinelHubInputTask(
            data_collection=collection,
            bands=["B01", "B02", "B03"],
            bands_feature=(FeatureType.DATA, "L1C_data"),
            additional_data=[(FeatureType.MASK, "dataMask")],  # cloud mask
            maxcc=0.8,
            resolution=r,
            config=config,  # important since we are using sentinel-dl, alternatively save ID and secret to default profile
        )
    # save = SaveTask("eopatch_fd")
    output_task = OutputTask("eopatch")
    workflow_nodes = linearly_connect_tasks(
        input_task,
        # save,
        output_task,
    )
    workflow = EOWorkflow(workflow_nodes)
    # workflow.dependency_graph() additionally install graphviz and eo-learn[VISUALIZATION]
    # take nodes, these are not the same as tasks!!!
    input_node = workflow_nodes[0]

    execution_args = [
        {
            input_node: {"bbox": bbox_list[0], "time_interval": None},
            # save_node: {"eopatch_folder": "eopatch_file2"},
        }
    ]

    executor = EOExecutor(workflow, execution_args, logs_folder="./reports")
    res = executor.run(workers=4)

    executor.make_report()

    failed_ids = executor.get_failed_executions()
    if failed_ids:
        raise RuntimeError(
            f"Execution failed EOPatches with IDs:\n{failed_ids}\n"
            f"For more info check report at {executor.get_report_path()}"
        )

    return res[0].outputs["eopatch"]

In [67]:
p1 = test(10, 512, None)

100%|██████████| 1/1 [00:04<00:00,  4.57s/it]
C:\Users\blazr\miniconda3\envs\sentinel_env\lib\site-packages\eolearn\visualization\eoexecutor.py:62: EOUserWarning: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH.
Please install the system package 'graphviz' (in addition to the python package) to have the dependency graph in the final report!
  warnings.warn(


In [68]:
p2 = test(20, 512, 5120)

Fixing bbox chunk size to 5120 m, resolution and patch_size are ignored.


100%|██████████| 1/1 [00:04<00:00,  4.46s/it]
C:\Users\blazr\miniconda3\envs\sentinel_env\lib\site-packages\eolearn\visualization\eoexecutor.py:62: EOUserWarning: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH.
Please install the system package 'graphviz' (in addition to the python package) to have the dependency graph in the final report!
  warnings.warn(


In [69]:
print(p1)
print(p2)

EOPatch(
  bbox=BBox(((368640.0, 5125120.0), (373760.0, 5130240.0)), crs=CRS('32633'))
  mask={
    dataMask: numpy.ndarray(shape=(1, 512, 512, 1), dtype=bool)
  }
  data={
    L1C_data: numpy.ndarray(shape=(1, 512, 512, 3), dtype=float32)
  }
)
EOPatch(
  bbox=BBox(((368640.0, 5125120.0), (373760.0, 5130240.0)), crs=CRS('32633'))
  mask={
    dataMask: numpy.ndarray(shape=(1, 512, 512, 1), dtype=bool)
  }
  data={
    L1C_data: numpy.ndarray(shape=(1, 512, 512, 3), dtype=float32)
  }
)


In [70]:
# should be the same
p1.bbox == p2.bbox

True

In [71]:
p3 = test(10, 1024, 2560)

Fixing bbox chunk size to 2560 m, resolution and patch_size are ignored.


100%|██████████| 1/1 [00:05<00:00,  5.65s/it]
C:\Users\blazr\miniconda3\envs\sentinel_env\lib\site-packages\eolearn\visualization\eoexecutor.py:62: EOUserWarning: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH.
Please install the system package 'graphviz' (in addition to the python package) to have the dependency graph in the final report!
  warnings.warn(


In [72]:
p3

EOPatch(
  bbox=BBox(((371200.0, 5125120.0), (373760.0, 5127680.0)), crs=CRS('32633'))
  mask={
    dataMask: numpy.ndarray(shape=(1, 1024, 1024, 1), dtype=bool)
  }
  data={
    L1C_data: numpy.ndarray(shape=(1, 1024, 1024, 3), dtype=float32)
  }
)

In [73]:
p4 = test(10, 256, None)

100%|██████████| 1/1 [00:03<00:00,  4.00s/it]
C:\Users\blazr\miniconda3\envs\sentinel_env\lib\site-packages\eolearn\visualization\eoexecutor.py:62: EOUserWarning: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH.
Please install the system package 'graphviz' (in addition to the python package) to have the dependency graph in the final report!
  warnings.warn(


In [74]:
# should be same region, but different res
p3.bbox == p4.bbox

True

In [75]:
fig, axs = plt.subplots(1, 2, figsize=(40, 20))

axs[0].imshow(np.clip(p3.data["L1C_data"][0][..., [2, 1, 0]], 0, 1))
axs[1].imshow(np.clip(p4.data["L1C_data"][0][..., [2, 1, 0]], 0, 1))
plt.savefig("comp_mega.png")

In [76]:
p4

EOPatch(
  bbox=BBox(((371200.0, 5125120.0), (373760.0, 5127680.0)), crs=CRS('32633'))
  mask={
    dataMask: numpy.ndarray(shape=(1, 256, 256, 1), dtype=bool)
  }
  data={
    L1C_data: numpy.ndarray(shape=(1, 256, 256, 3), dtype=float32)
  }
)

In [77]:
fig, axs = plt.subplots(1, 2)

axs[0].imshow(np.clip(p1.data["L1C_data"][0][..., [2, 1, 0]], 0, 1))
axs[1].imshow(np.clip(p2.data["L1C_data"][0][..., [2, 1, 0]], 0, 1))
plt.savefig("comp2.png")

In [78]:
# problematic stuff
p5 = test(10, 512, None)
# setting fixed size of bbox in meters and then passing resolution will impact patch pixel resolution.
# also keep in mind that this "resolution" enforced is the default behavior of actual code.
p6 = test(20, 512, 5120, force_res=20)

100%|██████████| 1/1 [00:04<00:00,  4.76s/it]
C:\Users\blazr\miniconda3\envs\sentinel_env\lib\site-packages\eolearn\visualization\eoexecutor.py:62: EOUserWarning: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH.
Please install the system package 'graphviz' (in addition to the python package) to have the dependency graph in the final report!
  warnings.warn(


Fixing bbox chunk size to 5120 m, resolution and patch_size are ignored.


100%|██████████| 1/1 [00:03<00:00,  3.96s/it]
C:\Users\blazr\miniconda3\envs\sentinel_env\lib\site-packages\eolearn\visualization\eoexecutor.py:62: EOUserWarning: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH.
Please install the system package 'graphviz' (in addition to the python package) to have the dependency graph in the final report!
  warnings.warn(


In [79]:
print(p5)
print(p6)

EOPatch(
  bbox=BBox(((368640.0, 5125120.0), (373760.0, 5130240.0)), crs=CRS('32633'))
  mask={
    dataMask: numpy.ndarray(shape=(1, 512, 512, 1), dtype=bool)
  }
  data={
    L1C_data: numpy.ndarray(shape=(1, 512, 512, 3), dtype=float32)
  }
)
EOPatch(
  bbox=BBox(((368640.0, 5125120.0), (373760.0, 5130240.0)), crs=CRS('32633'))
  mask={
    dataMask: numpy.ndarray(shape=(1, 256, 256, 1), dtype=bool)
  }
  data={
    L1C_data: numpy.ndarray(shape=(1, 256, 256, 3), dtype=float32)
  }
)


In [80]:
p5.bbox == p6.bbox

True

In [81]:
print(p5.timestamps)

None
